In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
os.chdir("D:/research")

more = True
use_drive = False
use_dev = True
use_cluster = True
use_sample = True
use_temporal= True
use_road = True
features = pd.read_csv('Device_all_cluster1.csv')
features = features.drop(['list'], axis = 1)
features.fillna({'HourMax1':'None', 'HourMax2':'None', 'HourMax3':'None', 'HourMax4':'None',
                 'CountMax1':0,  'CountMax2':0,  'CountMax3':0, 'CountMax4':0}, inplace=True)

features = pd.concat([features,pd.get_dummies(features['GeospatialType'])], axis = 1)

features['CountMax1'] = features['CountMax1']/features['TotalNumTrip']
features['CountMax2'] = features['CountMax2']/features['TotalNumTrip']
features['CountMax3'] = features['CountMax3']/features['TotalNumTrip']
features['CountMax4'] = features['CountMax4']/features['TotalNumTrip']


features['first_cluster_count'] = features['first_cluster_count']/features['TotalNumTrip']
features['second_cluster_count'] = features['second_cluster_count']/features['TotalNumTrip']
drop_dev = ['std_average_speed', 'std_detour_metrics', 'std_POInum', 'std_Total_time_interval', 'std_TotalDistance',
           'STD Local', 'STD Collector', 'STD Principal Arterial', 'STD Minor Arterial', 'STD None']
drop_road = ['Average Local', 'Average Collector', 'Average Principal Arterial',
             'Average Minor Arterial','Average None']
drop_cluster = ['first_cluster_count', 'second_cluster_count','total_cluster_num']
drop_sample = ['mean_AvgTimeGra','mean_AvgDistanceGra','mean_point_num']
drop_temporal = ['CountMax1','DistinctHour']
drop_drive = ['mean_detour_metrics', 'mean_POInum', 'mean_TotalDistance',
       'mean_average_speed', 'mean_Total_time_interval','EE', 'EI', 'IE', 'II']
# features = features.drop(drop_names, axis=1)
if use_dev == False:
    features = features.drop(drop_dev, axis=1)
if use_drive == False:
    features = features.drop(drop_drive, axis=1)
    
if use_cluster == False:
    features = features.drop(drop_cluster, axis=1)
if use_sample == False:
    features = features.drop(drop_sample, axis=1)
if use_road == False:
    features = features.drop(drop_road, axis=1)
if use_temporal == True:
    features = pd.concat([features,pd.get_dummies(features['HourMax1'])], axis = 1)
else:
    features = features.drop(drop_temporal, axis=1)
features = features.dropna()
if use_dev == True:
    features = features.dropna(axis = 0)
Consumer = features[features['min_profile'] == 1]
Taxi = features[features['min_profile'] == 2]
Delivery = features[features['min_profile'] == 3]
Truck = features[features['min_profile'] == 4]

if more == True:
    n = min([Consumer.shape[0], Delivery.shape[0], Truck.shape[0]])
    
    Truck = Truck.sample(n=n, random_state=33)
    Consumer = Consumer.sample(n=n, random_state=33)
    Delivery = Delivery.sample(n=n, random_state=33)
    features = pd.concat([Consumer, Delivery, Truck], axis = 0)
    del Consumer, Delivery, Truck
    
else:
    n = Taxi.shape[0]
    Truck = Truck.sample(n=n, random_state=3)
    Consumer = Consumer.sample(n=n, random_state=23)
    Delivery = Delivery.sample(n=n, random_state=57)
    features = pd.concat([Consumer, Taxi, Delivery, Truck], axis = 0)
    del Consumer, Taxi, Delivery, Truck
    
labels = np.array(features['min_profile'])
features = features.drop(['DeviceId', 'min_profile','max_profile', 'GeospatialType', 'TotalNumTrip', 
                          'HourMax1',  'std_AvgTimeGra',  'std_AvgDistanceGra', 'std_point_num', 
                          'HourMax2','HourMax3','HourMax4', 'CountMax2', 'CountMax3', 'CountMax4'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=40)
    

# from sklearn import preprocessing
# normalized_features = preprocessing.normalize(features)
# X_train, X_test, y_train, y_test = train_test_split(normalized_features, labels, test_size=0.2, random_state=40)

# X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=40)


In [11]:
features.columns

Index(['mean_detour_metrics', 'mean_POInum', 'mean_TotalDistance',
       'mean_average_speed', 'mean_Total_time_interval', 'first_cluster_count',
       'second_cluster_count', 'EE', 'EI', 'IE', 'II'],
      dtype='object')

In [4]:
features.shape

(72024, 55)

In [2]:
X_train.shape

(57619, 55)

In [3]:
X_test.shape

(14405, 55)

In [24]:
print(labels)

[4 4 2 ... 3 2 3]


In [7]:
features.head()

,mean_detour_metrics,mean_POInum,mean_TotalDistance,mean_average_speed,mean_Total_time_interval,EE,EI,IE,II,Average Local,Average Collector,Average Principal Arterial,Average Minor Arterial,Average None
0,0.992155,0.055556,4.749152,0.013728,404.555556,0,0,0,1,0.018519,0.157407,0.695370,0.009259,0.119444
1,0.991673,0.000000,2.537133,0.012897,240.375000,0,0,0,1,0.083333,0.109375,0.713542,0.000000,0.093750
2,0.582224,0.613360,17.355034,0.006699,2105.651269,0,0,0,1,0.144788,0.205418,0.417828,0.220518,0.011449
3,0.788698,1.915254,25.004719,0.008691,2781.372881,0,0,0,1,0.184953,0.114037,0.571453,0.105117,0.024441
4,0.973945,0.000000,5.144183,0.014282,396.771930,0,0,0,1,0.121053,0.003509,0.582749,0.035088,0.257602


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

clf = LogisticRegression(random_state=0, max_iter = 1000000)
param_grid = {
    'penalty': ['l2'],
    'C': [3,4,5],
    'solver': ['newton-cg', 'sag', 'saga', 'lbfgs'],
    'multi_class': ['multinomial']
}
# Cr
grid_search = GridSearchCV(estimator = clf, param_grid = param_grid, scoring='accuracy',
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
model = LogisticRegression(**best_params)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
accuracy = metrics.precision_score(y_test, predictions, average = 'micro')
precision = metrics.precision_score(y_test, predictions, average = None)
recall = metrics.recall_score(y_test, predictions, average = None)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed: 62.8min finished
C:\Users\wangr\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\wangr\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [9]:
np.unique(y_train, return_counts=True)

(array([1, 3, 4], dtype=int64), array([19255, 19228, 19136], dtype=int64))

In [10]:
np.unique(y_test, return_counts=True)

(array([1, 3, 4], dtype=int64), array([4753, 4780, 4872], dtype=int64))

In [3]:
best_params

{'C': 4, 'multi_class': 'multinomial', 'penalty': 'l2', 'solver': 'lbfgs'}

In [4]:
accuracy

0.7013536966331135

In [5]:
precision

array([0.89740495, 0.59680639, 0.69718449])

In [6]:
recall

array([0.62571008, 0.75062762, 0.72680624])

In [17]:
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# clf = SVC(gamma='auto')
# param_grid = {'kernel': ['linear'],
#                      'C': [1, 10, 100]}
                   

# grid_search = GridSearchCV(estimator = clf, param_grid = param_grid, scoring='accuracy',
#                           cv = 3, n_jobs = -1, verbose = 3)
# grid_search.fit(X_train, y_train)
# best_params = grid_search.best_params_
best_params = {'C': 1, 'kernel': 'linear'}
model = SVC(**best_params)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
accuracy = metrics.precision_score(y_test, predictions, average = 'micro')
precision = metrics.precision_score(y_test, predictions, average = None)
recall = metrics.recall_score(y_test, predictions, average = None)